# Лабораторная работа 4, Степанов 6408, 1 вариант

Tensorflow 2.x

1) Подготовка данных

2) Использование Keras Model API

3) Использование Keras Sequential + Functional API

https://www.tensorflow.org/tutorials

Для выполнения лабораторной работы необходимо установить tensorflow версии 2.0 или выше .

Рекомендуется использовать возможности Colab'а по обучению моделей на GPU.



In [1]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt

%matplotlib inline

# Подготовка данных
Загрузите набор данных из предыдущей лабораторной работы. 

In [2]:
def load_cifar10(num_training=49000, num_validation=1000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test

# If there are errors with SSL downloading involving self-signed certificates,
# it may be that your Python version was recently installed on the current machine.
# See: https://github.com/tensorflow/tensorflow/issues/10779
# To fix, run the command: /Applications/Python\ 3.7/Install\ Certificates.command
#   ...replacing paths as necessary.

# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int32
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [3]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y
        
        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [4]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

0 (64, 32, 32, 3) (64,)
1 (64, 32, 32, 3) (64,)
2 (64, 32, 32, 3) (64,)
3 (64, 32, 32, 3) (64,)
4 (64, 32, 32, 3) (64,)
5 (64, 32, 32, 3) (64,)
6 (64, 32, 32, 3) (64,)


#  Keras Model Subclassing API


Для реализации собственной модели с помощью Keras Model Subclassing API необходимо выполнить следующие шаги:

1) Определить новый класс, который является наследником tf.keras.Model.

2) В методе __init__() определить все необходимые слои из модуля tf.keras.layer

3) Реализовать прямой проход в методе call() на основе слоев, объявленных в __init__()

Ниже приведен пример использования keras API для определения двухслойной полносвязной сети. 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras

In [5]:
device = '/device:GPU:0'

In [6]:
class TwoLayerFC(tf.keras.Model):
    def __init__(self, hidden_size, num_classes):
        super(TwoLayerFC, self).__init__()        
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.fc1 = tf.keras.layers.Dense(hidden_size, activation='relu',
                                   kernel_initializer=initializer)
        self.fc2 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                   kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()
    
    def call(self, x, training=False):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def test_TwoLayerFC():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    x = tf.zeros((64, input_size))
    model = TwoLayerFC(hidden_size, num_classes)
    with tf.device(device):
        scores = model(x)
        print(scores.shape)
        
test_TwoLayerFC()

(64, 10)


Реализуйте трехслойную CNN для вашей задачи классификации. 

Архитектура сети:
    
1. Сверточный слой (5 x 5 kernels, zero-padding = 'same')
2. Функция активации ReLU 
3. Сверточный слой (3 x 3 kernels, zero-padding = 'same')
4. Функция активации ReLU 
5. Полносвязный слой 
6. Функция активации Softmax 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Conv2D

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dense

In [7]:
class ThreeLayerConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, num_classes):
        super(ThreeLayerConvNet, self).__init__()
        ########################################################################
        # TODO: Implement the __init__ method for a three-layer ConvNet. You   #
        # should instantiate layer objects to be used in the forward pass.     #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.conv1 = tf.keras.layers.Conv2D(channel_1, (5, 5), padding='same', activation='relu', kernel_initializer=initializer)
        self.conv2 = tf.keras.layers.Conv2D(channel_2, (3, 3), padding='same', activation='relu', kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax', kernel_initializer=initializer)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################
        
    def call(self, x, training=False):
        scores = None
        ########################################################################
        # TODO: Implement the forward pass for a three-layer ConvNet. You      #
        # should use the layer objects defined in the __init__ method.         #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        scores = self.conv1(x)
        scores = self.conv2(scores)
        scores = self.flatten(scores)
        scores = self.fc(scores)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################        
        return scores

In [8]:
def test_ThreeLayerConvNet():    
    channel_1, channel_2, num_classes = 12, 8, 10
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)
    with tf.device(device):
        x = tf.zeros((64, 3, 32, 32))
        scores = model(x)
        print(scores.shape)

test_ThreeLayerConvNet()

(64, 10)


Пример реализации процесса обучения:

In [9]:
def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the MNIST training set and periodically checks
    accuracy on the MNIST validation set.
    
    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for
    
    Returns: Nothing, but prints progress during trainingn
    """    
    with tf.device(device):
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
        
        model = model_init_fn()
        optimizer = optimizer_init_fn()
        
        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
    
        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')
        
        i = 0
        for epoch in range(num_epochs):
            
            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            train_loss.reset_state()
            train_accuracy.reset_state()
            
            for x_np, y_np in train_dset:
                with tf.GradientTape() as tape:
                    
                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)
      
                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                    
                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)
                    
                    if i % print_every == 0:
                        val_loss.reset_state()
                        val_accuracy.reset_state()
                        for test_x, test_y in val_dset:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)
                        
                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(i, epoch + 1,
                                             train_loss.result(),
                                             train_accuracy.result() * 100,
                                             val_loss.result(),
                                             val_accuracy.result() * 100))
                    i += 1

In [10]:
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2
print_every = 100

def model_init_fn():
    return TwoLayerFC(hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.807525157928467, Accuracy: 4.6875, Val Loss: 2.8451218605041504, Val Accuracy: 13.699999809265137
Iteration 100, Epoch 1, Loss: 2.2336370944976807, Accuracy: 28.217823028564453, Val Loss: 1.8982609510421753, Val Accuracy: 37.5
Iteration 200, Epoch 1, Loss: 2.0765202045440674, Accuracy: 31.91853141784668, Val Loss: 1.8570573329925537, Val Accuracy: 38.60000228881836
Iteration 300, Epoch 1, Loss: 2.0014705657958984, Accuracy: 33.850704193115234, Val Loss: 1.8679765462875366, Val Accuracy: 37.29999923706055
Iteration 400, Epoch 1, Loss: 1.933796763420105, Accuracy: 35.7465705871582, Val Loss: 1.7132222652435303, Val Accuracy: 41.29999923706055
Iteration 500, Epoch 1, Loss: 1.8893852233886719, Accuracy: 36.96669006347656, Val Loss: 1.6431317329406738, Val Accuracy: 43.20000076293945
Iteration 600, Epoch 1, Loss: 1.8591744899749756, Accuracy: 37.77298355102539, Val Loss: 1.7106341123580933, Val Accuracy: 42.20000076293945
Iteration 700, Epoch 1, Loss: 1.8346666

Обучите трехслойную CNN. В tf.keras.optimizers.SGD укажите Nesterov momentum = 0.9 . 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/optimizers/SGD

Значение accuracy на валидационной выборке после 1 эпохи обучения должно быть > 50% .

In [11]:
learning_rate = 3e-3
channel_1, channel_2, num_classes = 32, 16, 10

def model_init_fn():
    model = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return model

def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate, 0.9, nesterov=True)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.2870841026306152, Accuracy: 9.375, Val Loss: 3.582383871078491, Val Accuracy: 14.0
Iteration 100, Epoch 1, Loss: 1.9755779504776, Accuracy: 32.33292007446289, Val Loss: 1.6516742706298828, Val Accuracy: 43.20000076293945
Iteration 200, Epoch 1, Loss: 1.7890347242355347, Accuracy: 37.78762435913086, Val Loss: 1.4892115592956543, Val Accuracy: 46.79999923706055
Iteration 300, Epoch 1, Loss: 1.6885827779769897, Accuracy: 41.04547119140625, Val Loss: 1.4063520431518555, Val Accuracy: 50.70000457763672
Iteration 400, Epoch 1, Loss: 1.6092686653137207, Accuracy: 43.637001037597656, Val Loss: 1.3526923656463623, Val Accuracy: 52.70000076293945
Iteration 500, Epoch 1, Loss: 1.5523571968078613, Accuracy: 45.565120697021484, Val Loss: 1.294385313987732, Val Accuracy: 54.5
Iteration 600, Epoch 1, Loss: 1.5149117708206177, Accuracy: 46.84640121459961, Val Loss: 1.2912663221359253, Val Accuracy: 54.79999923706055
Iteration 700, Epoch 1, Loss: 1.4819101095199585, Accura

# Использование Keras Sequential API для реализации последовательных моделей.

Пример для полносвязной сети:

In [12]:
learning_rate = 1e-2

def model_init_fn():
    input_shape = (32, 32, 3)
    hidden_layer_size, num_classes = 4000, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu',
                              kernel_initializer=initializer),
        tf.keras.layers.Dense(num_classes, activation='softmax', 
                              kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    return model

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate) 

train_part34(model_init_fn, optimizer_init_fn)

C:\Users\_Asus_\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Iteration 0, Epoch 1, Loss: 2.81013560295105, Accuracy: 14.0625, Val Loss: 2.8950588703155518, Val Accuracy: 12.100000381469727
Iteration 100, Epoch 1, Loss: 2.235530376434326, Accuracy: 28.97586441040039, Val Loss: 1.919697642326355, Val Accuracy: 37.20000076293945
Iteration 200, Epoch 1, Loss: 2.0761966705322266, Accuracy: 32.57929229736328, Val Loss: 1.8408478498458862, Val Accuracy: 39.39999771118164
Iteration 300, Epoch 1, Loss: 2.0039479732513428, Accuracy: 34.24003601074219, Val Loss: 1.8919141292572021, Val Accuracy: 37.70000076293945
Iteration 400, Epoch 1, Loss: 1.9356919527053833, Accuracy: 36.0310173034668, Val Loss: 1.7334827184677124, Val Accuracy: 41.10000228881836
Iteration 500, Epoch 1, Loss: 1.890637993812561, Accuracy: 37.07584762573242, Val Loss: 1.6565459966659546, Val Accuracy: 43.20000076293945
Iteration 600, Epoch 1, Loss: 1.8603553771972656, Accuracy: 37.921173095703125, Val Loss: 1.7027995586395264, Val Accuracy: 41.400001525878906
Iteration 700, Epoch 1, Loss

Альтернативный менее гибкий способ обучения:

In [13]:
model = model_init_fn()
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

766/766 ━━━━━━━━━━━━━━━━━━━━ 34s 44ms/step - loss: 2.0163 - sparse_categorical_accuracy: 0.3364 - val_loss: 1.6836 - val_sparse_categorical_accuracy: 0.4280
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 1.7411 - sparse_categorical_accuracy: 0.4200


[1.7572673559188843, 0.4156999886035919]

Перепишите реализацию трехслойной CNN с помощью tf.keras.Sequential API . Обучите модель двумя способами.

In [14]:
def model_init_fn():
    model = None
    ############################################################################
    # TODO: Construct a three-layer ConvNet using tf.keras.Sequential.         #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    channel_1, channel_2, num_classes = 32, 16, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Input(shape=(32, 32, 3)),
        tf.keras.layers.Conv2D(channel_1, (5, 5), padding='same', activation='relu', kernel_initializer=initializer),
        tf.keras.layers.Conv2D(channel_2, (3, 3), padding='same', activation='relu', kernel_initializer=initializer),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_classes, activation='softmax', kernel_initializer=initializer)
    ]
    model = tf.keras.Sequential(layers)
    
    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                            END OF YOUR CODE                              #
    ############################################################################
    return model

learning_rate = 5e-4
def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate, momentum=0.9, nesterov=True)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.396503448486328, Accuracy: 18.75, Val Loss: 2.7217164039611816, Val Accuracy: 10.0
Iteration 100, Epoch 1, Loss: 2.031841278076172, Accuracy: 27.70730209350586, Val Loss: 1.8510446548461914, Val Accuracy: 35.400001525878906
Iteration 200, Epoch 1, Loss: 1.907713770866394, Accuracy: 32.52487564086914, Val Loss: 1.709571123123169, Val Accuracy: 41.400001525878906
Iteration 300, Epoch 1, Loss: 1.8330013751983643, Accuracy: 35.21075439453125, Val Loss: 1.6601403951644897, Val Accuracy: 42.69999694824219
Iteration 400, Epoch 1, Loss: 1.7693586349487305, Accuracy: 37.70651626586914, Val Loss: 1.6055705547332764, Val Accuracy: 43.79999923706055
Iteration 500, Epoch 1, Loss: 1.7241100072860718, Accuracy: 39.240272521972656, Val Loss: 1.5422197580337524, Val Accuracy: 45.29999923706055
Iteration 600, Epoch 1, Loss: 1.6940221786499023, Accuracy: 40.39621353149414, Val Loss: 1.5343222618103027, Val Accuracy: 46.5
Iteration 700, Epoch 1, Loss: 1.6665406227111816, Accu

In [15]:
model = model_init_fn()
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

766/766 ━━━━━━━━━━━━━━━━━━━━ 15s 20ms/step - loss: 1.7843 - sparse_categorical_accuracy: 0.3824 - val_loss: 1.4151 - val_sparse_categorical_accuracy: 0.5060
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.4511 - sparse_categorical_accuracy: 0.4854


[1.4483753442764282, 0.49160000681877136]

# Использование Keras Functional API

Для реализации более сложных архитектур сети с несколькими входами/выходами, повторным использованием слоев, "остаточными" связями (residual connections) необходимо явно указать входные и выходные тензоры. 

Ниже представлен пример для полносвязной сети. 

In [16]:
def two_layer_fc_functional(input_shape, hidden_size, num_classes):  
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    inputs = tf.keras.Input(shape=input_shape)
    flattened_inputs = tf.keras.layers.Flatten()(inputs)
    fc1_output = tf.keras.layers.Dense(hidden_size, activation='relu',
                                 kernel_initializer=initializer)(flattened_inputs)
    scores = tf.keras.layers.Dense(num_classes, activation='softmax',
                             kernel_initializer=initializer)(fc1_output)

    # Instantiate the model given inputs and outputs.
    model = tf.keras.Model(inputs=inputs, outputs=scores)
    return model

def test_two_layer_fc_functional():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    input_shape = (50,)
    
    x = tf.zeros((64, input_size))
    model = two_layer_fc_functional(input_shape, hidden_size, num_classes)
    
    with tf.device(device):
        scores = model(x)
        print(scores.shape)
        
test_two_layer_fc_functional()

(64, 10)


In [17]:
input_shape = (32, 32, 3)
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return two_layer_fc_functional(input_shape, hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.085843086242676, Accuracy: 12.5, Val Loss: 2.745102882385254, Val Accuracy: 13.09999942779541
Iteration 100, Epoch 1, Loss: 2.211057662963867, Accuracy: 28.97586441040039, Val Loss: 1.8858118057250977, Val Accuracy: 37.79999923706055
Iteration 200, Epoch 1, Loss: 2.05918288230896, Accuracy: 32.39272689819336, Val Loss: 1.8582072257995605, Val Accuracy: 38.20000076293945
Iteration 300, Epoch 1, Loss: 1.989976406097412, Accuracy: 34.18293380737305, Val Loss: 1.8867124319076538, Val Accuracy: 37.29999923706055
Iteration 400, Epoch 1, Loss: 1.9229463338851929, Accuracy: 36.00763702392578, Val Loss: 1.720579743385315, Val Accuracy: 41.20000076293945
Iteration 500, Epoch 1, Loss: 1.8794021606445312, Accuracy: 37.103919982910156, Val Loss: 1.661085844039917, Val Accuracy: 43.0
Iteration 600, Epoch 1, Loss: 1.8494094610214233, Accuracy: 37.965370178222656, Val Loss: 1.6953469514846802, Val Accuracy: 42.099998474121094
Iteration 700, Epoch 1, Loss: 1.82449054718017

Поэкспериментируйте с архитектурой сверточной сети. Для вашего набора данных вам необходимо получить как минимум 70% accuracy на валидационной выборке за 10 эпох обучения. Опишите все эксперименты и сделайте выводы (без выполнения данного пункта работы приниматься не будут). 

Эспериментируйте с архитектурой, гиперпараметрами, функцией потерь, регуляризацией, методом оптимизации.  

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/BatchNormalization#methods https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dropout#methods

In [18]:
class CustomConvNet_with_pool(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet_with_pool, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        channels, num_classes = [32, 64], [100, 10]
        
        initializer = tf.initializers.VarianceScaling(scale=2.0)

        # Определение сверточных слоев
        self.conv1 = tf.keras.layers.Conv2D(channels[0], (5, 5), (3, 3), padding='same', kernel_initializer=initializer, input_shape=(32, 32, 3))
        self.conv2 = tf.keras.layers.Conv2D(channels[1], (3, 3), (3, 3), padding='same', kernel_initializer=initializer)
        self.pool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))
        
        # Слой для развертывания
        self.fl = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(num_classes[0], activation='relu', kernel_initializer=initializer)  
        self.fc2 = tf.keras.layers.Dense(num_classes[1], activation='softmax', kernel_initializer=initializer)             

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
    
    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.conv1(input_tensor)        
        x = self.conv2(x)
        x = self.pool(x)

        x = self.fl(x)
        x = self.fc1(x)
        x = self.fc2(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        return x


print_every = 700
num_epochs = 6

model = CustomConvNet_with_pool()

def model_init_fn():
    return CustomConvNet_with_pool()

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate) 

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

C:\Users\_Asus_\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Iteration 0, Epoch 1, Loss: 5.302567958831787, Accuracy: 12.5, Val Loss: 4.010174751281738, Val Accuracy: 11.800000190734863
Iteration 700, Epoch 1, Loss: 1.7687824964523315, Accuracy: 37.8521728515625, Val Loss: 1.5573753118515015, Val Accuracy: 44.80000305175781
Iteration 1400, Epoch 2, Loss: 1.4606837034225464, Accuracy: 47.947837829589844, Val Loss: 1.4408284425735474, Val Accuracy: 47.29999923706055
Iteration 2100, Epoch 3, Loss: 1.3456016778945923, Accuracy: 52.03207015991211, Val Loss: 1.3914951086044312, Val Accuracy: 50.19999694824219
Iteration 2800, Epoch 4, Loss: 1.260055422782898, Accuracy: 55.100643157958984, Val Loss: 1.3200856447219849, Val Accuracy: 51.20000076293945
Iteration 3500, Epoch 5, Loss: 1.198090672492981, Accuracy: 57.63372039794922, Val Loss: 1.2911622524261475, Val Accuracy: 54.10000228881836
Iteration 4200, Epoch 6, Loss: 1.1465669870376587, Accuracy: 59.492919921875, Val Loss: 1.2711378335952759, Val Accuracy: 55.599998474121094


In [19]:
def model_init_fn():
    return CustomConvNet_with_pool()

# Let's try SGD optimizer
def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.SGD(learning_rate)

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 4.316521644592285, Accuracy: 9.375, Val Loss: 4.7897725105285645, Val Accuracy: 6.099999904632568
Iteration 700, Epoch 1, Loss: 2.3670122623443604, Accuracy: 19.699535369873047, Val Loss: 2.087339401245117, Val Accuracy: 23.0
Iteration 1400, Epoch 2, Loss: 2.025083303451538, Accuracy: 27.207183837890625, Val Loss: 1.9645044803619385, Val Accuracy: 28.299999237060547
Iteration 2100, Epoch 3, Loss: 1.9332650899887085, Accuracy: 30.76120376586914, Val Loss: 1.8954578638076782, Val Accuracy: 31.099998474121094
Iteration 2800, Epoch 4, Loss: 1.8764898777008057, Accuracy: 32.98645782470703, Val Loss: 1.8504605293273926, Val Accuracy: 33.5
Iteration 3500, Epoch 5, Loss: 1.8343899250030518, Accuracy: 34.721824645996094, Val Loss: 1.8103399276733398, Val Accuracy: 35.20000076293945
Iteration 4200, Epoch 6, Loss: 1.8040720224380493, Accuracy: 35.91644287109375, Val Loss: 1.7782799005508423, Val Accuracy: 35.29999923706055


In [20]:
class CustomConvNet(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        channels, num_classes = [64, 32], 10
        
        initializer = tf.initializers.VarianceScaling(scale=2.0)

        # Определение сверточных слоев
        self.conv1 = tf.keras.layers.Conv2D(channels[0], (5, 5), (1, 1), padding='same', kernel_initializer=initializer, input_shape=(32, 32, 3))
        self.conv2 = tf.keras.layers.Conv2D(channels[1], (3, 3), (1, 1), padding='same', kernel_initializer=initializer)

        # Нормализация
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.bn2 = tf.keras.layers.BatchNormalization()

        # Определение функции активации        
        self.relu1 = tf.keras.layers.ReLU()
        self.relu2 = tf.keras.layers.ReLU()
        
        self.dp1 = tf.keras.layers.Dropout(0.1)
        self.dp2 = tf.keras.layers.Dropout(0.1)

        self.pool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))

        # Слой для развертывания
        self.fl = tf.keras.layers.Flatten()       
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax', kernel_initializer=initializer)             

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
    
    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.conv1(input_tensor)        
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.dp1(x, training=training)
        x = self.pool(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.dp2(x, training=training)

        x = self.fl(x)
        x = self.fc(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        return x


print_every = 700
num_epochs = 10

model = CustomConvNet()

def model_init_fn():
    return CustomConvNet()

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate) 

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 2.589946746826172, Accuracy: 7.8125, Val Loss: 3.2146408557891846, Val Accuracy: 12.0
Iteration 700, Epoch 1, Loss: 1.3923286199569702, Accuracy: 52.1197395324707, Val Loss: 1.223808765411377, Val Accuracy: 57.20000457763672
Iteration 1400, Epoch 2, Loss: 0.9954715967178345, Accuracy: 65.32479858398438, Val Loss: 1.0613938570022583, Val Accuracy: 64.60000610351562
Iteration 2100, Epoch 3, Loss: 0.8661055564880371, Accuracy: 70.16146850585938, Val Loss: 0.9599947333335876, Val Accuracy: 66.5999984741211
Iteration 2800, Epoch 4, Loss: 0.7836273908615112, Accuracy: 73.055419921875, Val Loss: 0.9912348389625549, Val Accuracy: 65.69999694824219
Iteration 3500, Epoch 5, Loss: 0.7249841094017029, Accuracy: 74.87843322753906, Val Loss: 0.9470499157905579, Val Accuracy: 68.4000015258789
Iteration 4200, Epoch 6, Loss: 0.6773635745048523, Accuracy: 76.70147705078125, Val Loss: 0.9566137194633484, Val Accuracy: 68.0999984741211
Iteration 4900, Epoch 7, Loss: 0.629531979

Опишите все эксперименты, результаты. Сделайте выводы.

1. Среди оптимизаторов SGD и Adam, лучше себя показал `Adam`, будучи протестированы на одинаковых настройках.
2. Были исследованы несколько архитектур сетей, состоящих из:
   - a. сверточных слоев, нормализации, функций активации, слоев *dropout*, слоя для развертывания и полносвязного слоя;
   - b. сверточных слоев, максимального пулинга для первых двух слоев, слоя для развертывания и полносвязных слоев.
3. В дополнение к пункту выше стоит отметить, что добавление батч нормализации, дропаутов и дополнительного слоя значительно улучшило точность почти на $11$%. Дропауты оказали наибольшее влияние на результат, около $6$%.
4. Функция активации *sigmoid* показала себя хуже всех, гораздо лучшие результаты получены при использовании функции *ReLU*.
5. Для проведения различных эксперементов необходимо большое количество времени.
6. По итогу, при уменьшении аргумента дропаута с 0.2 до 0.1 на определенных параметрах была достигнута точность $83$% на тестовой выборке и $71$% на валидационной.